 **Import the Libraries**

In [34]:
import requests
import pandas as pd
import numpy as np
import random
!pip install geopy
from geopy.geocoders import Nominatim
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas.io.json import json_normalize
! pip install folium==0.5.0
import folium

You are using pip version 10.0.1, however version 21.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


  Running setup.py bdist_wheel for folium: started
  Running setup.py bdist_wheel for folium: finished with status 'done'
  Stored in directory: C:\Users\LENOVO\AppData\Local\pip\Cache\wheels\f8\98\ff\954791afc47740d554f0d9e5885fa09dd60c2265d42578e665
Successfully built folium


You are using pip version 10.0.1, however version 21.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


**Webpage Scraping**

In [2]:
req=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
table_class='wikitable'
req.status_code

200

In [3]:
soup=BeautifulSoup(req.text,'html.parser')
table=soup.find('table',{'class':'wikitable'})

In [4]:
table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td

In [5]:
type(table)

bs4.element.Tag

In [6]:
df=pd.read_html(str(table))

**Preparing the DataFrame**

In [7]:
df=pd.DataFrame(df[0])

In [8]:
df.head()

,0,1,2
0,Postal Code,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


Setting up the column names

In [9]:
df.columns=['Postal Code','Borough','Neighbourhood']

In [10]:
df.head(10)

,Postal Code,Borough,Neighbourhood
0,Postal Code,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M8A,Not assigned,Not assigned
9,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"


In [11]:
df=df.iloc[1:]

In [12]:
df.head()

,Postal Code,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [13]:
df.reset_index(drop=True,inplace=True)

In [14]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


**Removing all the 'Not Assigned' Boroughs**

In [15]:
df['Borough'].replace('Not assigned',np.nan,inplace=True)

In [16]:
df.shape

(180, 3)

In [17]:
df.dropna(subset=['Borough'],axis=0,inplace=True)

In [18]:
df.shape

(103, 3)

In [19]:
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [20]:
df.reset_index(drop=True,inplace=True)

In [21]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [22]:
df['Postal Code'].dtypes

dtype('O')

In [23]:
df.loc[1,:]

Postal Code                   M4A
Borough                North York
Neighbourhood    Victoria Village
Name: 1, dtype: object

**Assigning the Borough name to the Neighbourhood for all those Neighbourhood having Not Assinged value**

In [24]:
for i in range(df.shape[0]):
    if df.loc[i,:]['Neighbourhood']=='Not assigned':
        df.loc[i,:]['Neighbourhood']=df.loc[i,:]['Borough']

**Shape of the final DataFrame**

In [25]:
df.shape

(103, 3)

**Reading the Geographical Coordinates of all the Postal Codes**

In [26]:
df_coord=pd.read_csv('Geospatial_Coordinates.csv')

In [27]:
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [28]:
df_coord.shape

(103, 3)

In [29]:
df_coord['Postal Code'].dtypes

dtype('O')

**Merging two DataFrames to get the desired output DataFrame**

In [35]:
df_tor=pd.merge(df,df_coord,on='Postal Code')

In [36]:
df_tor.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


**Use geopy library to get the latitude and longitude values of Toronto**

In [37]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent='assignment')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of the City of Toronto are 43.6534817, -79.3839347.


**Create a map of Toronto with neighborhoods superimposed on top**

In [39]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(df_tor['Latitude'], df_tor['Longitude'], df_tor['Borough'], df_tor['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

**Foursquare Credentials and Version**

In [40]:
CLIENT_ID = 'client-ID' # your Foursquare ID
CLIENT_SECRET = 'client-secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: client-ID
CLIENT_SECRET:client-secret


**Selecting all the Neighborhoods of Toronto**

In [47]:
toronto_city=df_tor[df_tor['Borough'].str.contains('Toronto')].reset_index(drop=True)

In [48]:
toronto_city.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [49]:
toronto_city.shape

(40, 5)

**Explore the first neighborhood of Toronto in our dataframe**

In [50]:
toronto_city.loc[0,'Neighbourhood']

'Regent Park, Harbourfront'

**Check the Latitude and Longitude of the neighbourhood**

In [52]:
neighbour_lat=toronto_city.loc[0,'Latitude']
neighbour_lon=toronto_city.loc[0,'Longitude']
neighbour_name=toronto_city.loc[0,'Neighbourhood']
print('Latitude and Longitude of {} are {},{}'.format(neighbour_name,neighbour_lat,neighbour_lon))

Latitude and Longitude of Regent Park, Harbourfront are 43.6542599,-79.3606359
